In [1]:
import os

import json
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from typing import Optional, List
import datetime
from langchain_core.pydantic_v1 import BaseModel, Field
from serpapi import GoogleSearch

from dotenv import load_dotenv

_ = load_dotenv()

## 1. Initialize objects

In [2]:
llm = ChatOpenAI(model="gpt-4o")

## 2. Creation travel plan

In [23]:
GPTRAVEL_PROMPT_TRAVEL_PLAN_TEMPLATE = """
You are an intelligent travel planner.

The user is interested to travel to {destination} between {departure_date} and {return_date}. 
In addition to that the user suggested this for his/her travel: "{travel_text}".
With this in mind, generate a JSON that could contain a travel plan that could match user interests.
In the travel plan, please avoid activties related to flights and transports, also everything included about the accomodation and the restaurants. 
Just tell what to do when the user is at destination.
"""

class Activity(BaseModel):
    """Activity"""
    time: datetime.datetime = Field(description='Approximate time of the activicty. The format should be YYYY-MM-DD HH:MM:SS')
    place: str              = Field(description='Name of the place where is located the activicty')
    description: str        = Field(description='Description of the activicty')

class DayPlan(BaseModel):
    """Plan of a day in the travel plan"""
    date: datetime.date          = Field(description="Date of the plan.")
    activities: List[Activity]   = Field(description="List of activities to do in the day plan.")

class TravelPlan(BaseModel):
    """Travel plan"""
    
    day_plans: List[DayPlan] = Field(description="List of day plans activities.")

print(GPTRAVEL_PROMPT_TRAVEL_PLAN_TEMPLATE)


You are an intelligent travel planner.

The user is interested to travel to {destination} between {departure_date} and {return_date}. 
In addition to that the user suggested this for his/her travel: "{travel_text}".
With this in mind, generate a JSON that could contain a travel plan that could match user interests.
In the travel plan, please avoid activties related to flights and transports, also everything included about the accomodation and the restaurants. 
Just tell what to do when the user is at destination.



In [24]:
prompt_template = PromptTemplate(
    input_variables=["destination", "departure_date", "return_date", "travel_text"],
    template=GPTRAVEL_PROMPT_TRAVEL_PLAN_TEMPLATE
)


structured_llm = llm.with_structured_output(TravelPlan)

In [25]:
user_input = {
    "destination": "Paris",
    "departure_date": "2024-10-31",
    "return_date": "2024-11-02",
    "travel_text": "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night."
}

user_input

{'destination': 'Paris',
 'departure_date': '2024-10-31',
 'return_date': '2024-11-02',
 'travel_text': "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night."}

In [26]:
print(
    prompt_template.format(
        **user_input
    )
)


You are an intelligent travel planner.

The user is interested to travel to Paris between 2024-10-31 and 2024-11-02. 
In addition to that the user suggested this for his/her travel: "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night.".
With this in mind, generate a JSON that could contain a travel plan that could match user interests.
In the travel plan, please avoid activties related to flights and transports, also everything included about the accomodation and the restaurants. 
Just tell what to do when the user is at destination.



In [27]:
travel_plan_output = structured_llm.invoke(
    prompt_template.format(**user_input)
)

In [29]:
print(travel_plan_output.json())

{"day_plans": [{"date": "2024-10-31", "activities": [{"time": "2024-10-31T10:00:00", "place": "Eiffel Tower", "description": "Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."}, {"time": "2024-10-31T12:00:00", "place": "Champs-\u00c9lys\u00e9es", "description": "Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}, {"time": "2024-10-31T15:00:00", "place": "Louvre Museum", "description": "Visit the world's largest art museum and a historic monument in Paris. Don't miss the Mona Lisa and the Venus de Milo."}, {"time": "2024-10-31T18:00:00", "place": "Seine River", "description": "Take a romantic evening cruise on the Seine River. Enjoy the beautiful views of the city's landmarks illuminated at night."}]}, {"date": "2024-11-01", "activities": [{"time": "2024-11-01T10:00:00", "place": "Montmartre", "description": "Explore the charming district 

## 3a. From travel plan to check flights

In [38]:
GPTRAVEL_PROMPT_FLIGHT_QUERY_TEMPLATE = """
You are an intelligent travel planner.

The user is departing from {departure} and he plans to go to {destination}. 
You have previously generated the following travel plan: {travel_plan}

Given this in mind, we need to get the information required for calling SerpAPI for the search flights. 
Important: the dates of travel should be the first day of the travel plan for the outbound flight and the last day of the travel plan for the return flight.
You must give in output two inputs for the API, one for the outbound flight and the other for the return flight.
"""

DEPARTURE_ID_DESCRIPTION = """Parameter defines the departure airport code or location kgmid.
An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA.
For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.
A location kgmid is a string that starts with /m/. You can search for a location on Wikidata and use its "Freebase ID" as the location kgmid. For example, /m/0vzm is the location kgmid for Austin, TX.
You can specify multiple departure airports by separating them with a comma. For example, CDG,ORY,/m/04jpl
"""

ARRIVAL_ID_DESCRIPTION = """Parameter defines the arrival airport code or location kgmid.
An airport code is an uppercase 3-letter code. You can search for it on Google Flights or IATA.
For example, CDG is Paris Charles de Gaulle Airport and AUS is Austin-Bergstrom International Airport.
A location kgmid is a string that starts with /m/. You can search for a location on Wikidata and use its "Freebase ID" as the location kgmid. For example, /m/0vzm is the location kgmid for Austin, TX.
You can specify multiple arrival airports by separating them with a comma. For example, CDG,ORY,/m/04jpl
"""


class FlightSearchParameters(BaseModel):
    """Flight Search parameters"""

    departure_id: str = Field(description=DEPARTURE_ID_DESCRIPTION)
    arrival_id: str   = Field(description=ARRIVAL_ID_DESCRIPTION)

    date: datetime.date = Field(description="Date of the flight")


class SearchInputs(BaseModel):
    """Search Inputs for Travel"""
    outbound_input: FlightSearchParameters = Field(description='flight search parameters for the outbound flight')
    return_input: FlightSearchParameters   = Field(description='flight search parameters for the return flight')



print(GPTRAVEL_PROMPT_FLIGHT_QUERY_TEMPLATE)


You are an intelligent travel planner.

The user is departing from {departure} and he plans to go to {destination}. 
You have previously generated the following travel plan: {travel_plan}

Given this in mind, we need to get the information required for calling SerpAPI for the search flights. 
Important: the dates of travel should be the first day of the travel plan for the outbound flight and the last day of the travel plan for the return flight.
You must give in output two inputs for the API, one for the outbound flight and the other for the return flight.



In [39]:
llm_search_flight = ChatOpenAI(model="gpt-4o")


prompt_template_search_flights = PromptTemplate(
    input_variables=["departure", "destination", "travel_plan"],
    template=GPTRAVEL_PROMPT_FLIGHT_QUERY_TEMPLATE
)


structured_llm_search_flights = llm_search_flight.with_structured_output(SearchInputs)

In [40]:
params_search_flight = {
    "departure": "Milan",
    "destination": "Paris",
    "travel_plan": travel_plan_output.json()
}

print(prompt_template_search_flights.format(**params_search_flight))


You are an intelligent travel planner.

The user is departing from Milan and he plans to go to Paris. 
You have previously generated the following travel plan: {"day_plans": [{"date": "2024-10-31", "activities": [{"time": "2024-10-31T10:00:00", "place": "Eiffel Tower", "description": "Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."}, {"time": "2024-10-31T12:00:00", "place": "Champs-\u00c9lys\u00e9es", "description": "Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}, {"time": "2024-10-31T15:00:00", "place": "Louvre Museum", "description": "Visit the world's largest art museum and a historic monument in Paris. Don't miss the Mona Lisa and the Venus de Milo."}, {"time": "2024-10-31T18:00:00", "place": "Seine River", "description": "Take a romantic evening cruise on the Seine River. Enjoy the beautiful views of the city's landmarks illum

In [41]:
search_flights_input_query_output = structured_llm_search_flights.invoke(
    prompt_template_search_flights.format(**params_search_flight)
)

In [42]:
search_flighhts_inputs = json.loads(search_flights_input_query_output.json())

search_flighhts_inputs

{'outbound_input': {'departure_id': 'MXP',
  'arrival_id': 'CDG',
  'date': '2024-10-31'},
 'return_input': {'departure_id': 'CDG',
  'arrival_id': 'MXP',
  'date': '2024-11-02'}}

In [43]:
search_flight_result = {}

for flight_name, search_flighhts_input in search_flighhts_inputs.items():
    
    params = {
      "engine": "google_flights",
      "departure_id": search_flighhts_input['departure_id'],
      "arrival_id": search_flighhts_input['arrival_id'],
      "outbound_date": search_flighhts_input['date'],
      "currency": "EUR",
      "type": "2",
      "api_key": os.environ["SERPAPI_KEY"]
    }

    search = GoogleSearch(params)
    search_flight_result[flight_name] = search.get_dict()

In [44]:
search_flight_result['outbound_input']

{'search_metadata': {'id': '66d318d4d16915b8a69d66ee',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d4d16915b8a69d66ee.json',
  'created_at': '2024-08-31 13:21:24 UTC',
  'processed_at': '2024-08-31 13:21:24 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=EUR&tfs=CBwQAhoeEgoyMDI0LTEwLTMxagcIARIDTVhQcgcIARIDQ0RHQgEBSAFwAZgBAg==',
  'raw_html_file': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d4d16915b8a69d66ee.html',
  'prettify_html_file': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d4d16915b8a69d66ee.prettify',
  'total_time_taken': 1.27},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'type': '2',
  'departure_id': 'MXP',
  'arrival_id': 'CDG',
  'outbound_date': '2024-10-31',
  'currency': 'EUR'},
 'best_flights': [{'flights': [{'departure_airport': {'name': 'Milan Malpensa Airport',
      'id': 'MXP',
      'time': '2024-10-31 15:45'},
     'ar

In [45]:
search_flight_result['return_input']

{'search_metadata': {'id': '66d318d61a5b54adb4316375',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d61a5b54adb4316375.json',
  'created_at': '2024-08-31 13:21:26 UTC',
  'processed_at': '2024-08-31 13:21:26 UTC',
  'google_flights_url': 'https://www.google.com/travel/flights?hl=en&gl=us&curr=EUR&tfs=CBwQAhoeEgoyMDI0LTExLTAyagcIARIDQ0RHcgcIARIDTVhQQgEBSAFwAZgBAg==',
  'raw_html_file': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d61a5b54adb4316375.html',
  'prettify_html_file': 'https://serpapi.com/searches/bb6e505c312607ec/66d318d61a5b54adb4316375.prettify',
  'total_time_taken': 1.64},
 'search_parameters': {'engine': 'google_flights',
  'hl': 'en',
  'gl': 'us',
  'type': '2',
  'departure_id': 'CDG',
  'arrival_id': 'MXP',
  'outbound_date': '2024-11-02',
  'currency': 'EUR'},
 'best_flights': [{'flights': [{'departure_airport': {'name': 'Paris Charles de Gaulle Airport',
      'id': 'CDG',
      'time': '2024-11-02 06:05'},

In [ ]:
### parse travel plan ---> partenza da Milano (MXP, LIN, BGY) a Berlino (BER) -- andata e ritorno date
### chiamata api -----> lista opzioni voli [.....]
### lista opzioni, travel_plan -----> LLM ------> enriched travel plan (sui voli ora e prezzo)  

## 3b. From travel plan to check hotels 

In [58]:
GPTRAVEL_PROMPT_HOTEL_QUERY_TEMPLATE = """
You are an intelligent travel planner.

The user is departing plans to go to {destination}. This is what the user expressed: "{travel_text}".
You have previously generated the following travel plan: {travel_plan}
Given this in mind, we need to get the information required for calling SerpAPI for the search hotel accomodations. 
For each city of the plan, give in output the search api requests for each night of the travel. 

Important; if a city is comprised in one or more days in day_plans, just create one query input for that.
"""

class HotelSearchParameters(BaseModel):
    """Hotel Search parameters"""
    q: str = Field(description="Parameter defines the city of the accodomation. You can use anything that you would use in a regular Google Hotels search.")
    check_in_date: datetime.date = Field(description="Parameter defines the check-in date. The format is YYYY-MM-DD. e.g. 2024-08-31")
    check_out_date: datetime.date = Field(description="Parameter defines the check-out date. The format is YYYY-MM-DD. e.g. 2024-09-01")
    adults: Optional[int] = Field(description="Parameter defines the number of adults. Default to 2.")
    children: Optional[int] = Field(description="Parameter defines the number of children. Default to 0.")
    children_ages: Optional[List[int]] = Field(description="Parameter defines the ages of children. The age range is from 1 to 17, with children who haven't reached 1 year old being considered as 1. Example for single child only 5. Example for multiple children (seperated by comma ,): [5,8,10]")
    max_price: Optional[int] = Field(description="Parameter defines the upper bound of price range.")
    

class HotelSearchInputs(BaseModel):
    """Search Inputs for Travel accomodations"""
    search_list: List[HotelSearchParameters] = Field(description='List of hotel search parameters for each travel night')
    
    

In [59]:
llm_search_hotel = ChatOpenAI(model="gpt-4o")


prompt_template_search_hotel = PromptTemplate(
    input_variables=["destination", "travel_text", "travel_plan"],
    template=GPTRAVEL_PROMPT_HOTEL_QUERY_TEMPLATE
)


structured_llm_search_hotel = llm_search_hotel.with_structured_output(HotelSearchInputs)

In [60]:
params_search_hotel = {
    "destination": user_input['destination'],
    "travel_text": user_input['travel_text'],
    "travel_plan": travel_plan_output.json()
}

print(prompt_template_search_hotel.format(**params_search_hotel))


You are an intelligent travel planner.

The user is departing plans to go to Paris. This is what the user expressed: "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night.".
You have previously generated the following travel plan: {"day_plans": [{"date": "2024-10-31", "activities": [{"time": "2024-10-31T10:00:00", "place": "Eiffel Tower", "description": "Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."}, {"time": "2024-10-31T12:00:00", "place": "Champs-\u00c9lys\u00e9es", "description": "Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}, {"time": "2024-10-31T15:00:00", "place": "Louvre Museum", "description": "Visit the world's largest art museum and a historic monu

In [61]:
search_flights_input_hotels_output = structured_llm_search_hotel.invoke(
    prompt_template_search_hotel.format(**params_search_hotel)
)

In [62]:
search_flights_input_hotels_output

HotelSearchInputs(search_list=[HotelSearchParameters(q='Paris', check_in_date=datetime.date(2024, 10, 31), check_out_date=datetime.date(2024, 11, 2), adults=2, children=None, children_ages=None, max_price=300)])

In [63]:
search_hotels_inputs = json.loads(search_flights_input_hotels_output.json())

search_hotels_inputs

{'search_list': [{'q': 'Paris',
   'check_in_date': '2024-10-31',
   'check_out_date': '2024-11-02',
   'adults': 2,
   'children': None,
   'children_ages': None,
   'max_price': 300}]}

In [75]:
FIXED_HOTEL_PARAMS = {
    "currency": "EUR",
    "sort_by": "3",
    "api_key": os.environ["SERPAPI_KEY"],
    "engine": "google_hotels"
}

hotel_queries = []

for queries in search_hotels_inputs['search_list']:
    
    hotel_search_params = queries | FIXED_HOTEL_PARAMS

    hotel_search = GoogleSearch(hotel_search_params)
    hotel_search_results = hotel_search.get_dict()

    hotel_queries.append(hotel_search_results)

In [223]:
hotel_queries[0]

{'search_metadata': {'id': '66d3338ba8827832a4c10a33',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/7a6f42623ecab92a/66d3338ba8827832a4c10a33.json',
  'created_at': '2024-08-31 15:15:23 UTC',
  'processed_at': '2024-08-31 15:15:23 UTC',
  'google_hotels_url': 'https://www.google.com/_/TravelFrontendUi/data/batchexecute?rpcids=AtySUc&source-path=/travel/search&hl=en&gl=us&rt=c&soc-app=162&soc-platform=1&soc-device=1',
  'raw_html_file': 'https://serpapi.com/searches/7a6f42623ecab92a/66d3338ba8827832a4c10a33.html',
  'prettify_html_file': 'https://serpapi.com/searches/7a6f42623ecab92a/66d3338ba8827832a4c10a33.prettify',
  'total_time_taken': 2.04},
 'search_parameters': {'engine': 'google_hotels',
  'q': 'Paris',
  'gl': 'us',
  'hl': 'en',
  'currency': 'EUR',
  'check_in_date': '2024-10-31',
  'check_out_date': '2024-11-02',
  'adults': 2,
  'children': 0,
  'sort_by': '3',
  'max_price': '300'},
 'brands': [{'id': 33,
   'name': 'Accor Live Limitless',
   'c

In [87]:
hotel_queries[0]['properties'][0].keys()

dict_keys(['type', 'name', 'description', 'link', 'gps_coordinates', 'check_in_time', 'check_out_time', 'rate_per_night', 'total_rate', 'nearby_places', 'hotel_class', 'extracted_hotel_class', 'images', 'overall_rating', 'reviews', 'ratings', 'location_rating', 'reviews_breakdown', 'amenities', 'property_token', 'serpapi_property_details_link'])

In [ ]:
### parse travel plan ---> cities soggiorno persone
### chiamata api -----> lista opzioni hotel [.....]
### lista opzioni, travel_plan -----> LLM ------> enriched travel plan (soggiorni)  

## 3c. From api results to enriched travel plan

In [212]:
GPTRAVEL_PROMPT_PARSER_TEMPLATE = """
You are an intelligent travel planner.

The user is interested to travel to {destination} between {departure_date} and {return_date}. 
In addition to that the user suggested this for his/her travel: "{travel_text}".

In the previous iterations, you did the following things:

- Created the following travel plan: {travel_plan}

- Checked on Google Flights the best outbound flight: {outbound_flights_options}
- Checked on Google Flights the best return flight: {return_flights_options}

- Checked on Google Hotels all possible accomodations for each travel day: {accomodation_options}

Given this in mind, we need to find the best combinations of flights and accomodation that could better fit user interests. 
You will give in output an enriched travel plan, where you will tell the activities to do and which flights to take and in which place to sleep and stay. 

Remember: 

- outbound flight details should coincide with the first date of travel .Remeber that outbound flight time should fit the time of activities. Re adapt the travel plan to fit the flight.
- return flight details should coincide with the first date of travel. Remeber that return flight time should fit the time of activities. Re adapt the travel plan to fit the flight.
- hotel details is a list of accomodations that should cover the whole travel dates. To find a good place, take into consideration what the user whant and his budget.
"""

In [213]:
class GpsCoordinates(BaseModel):
    latitude: float = Field(description="The latitude coordinate of the hotel.")
    longitude: float = Field(description="The longitude coordinate of the hotel.")

class Rate(BaseModel):
    lowest: str = Field(description="The lowest rate displayed as a string (including currency symbol).")
    extracted_lowest: int = Field(description="The extracted lowest rate as a numerical value.")
    before_taxes_fees: str = Field(description="The rate before taxes and fees, displayed as a string (including currency symbol).")
    extracted_before_taxes_fees: int = Field(description="The extracted rate before taxes and fees as a numerical value.")

class Transportation(BaseModel):
    type_: str = Field(description="Type of transportation (e.g., Taxi, Walking).")
    duration: str = Field(description="Estimated duration for the transportation option.")

class NearbyPlace(BaseModel):
    name: str = Field(description="Name of the nearby place.")
    transportations: List[Transportation] = Field(description="Available transportation options and their durations to the place.")

class Rating(BaseModel):
    stars: conint(ge=1, le=5) = Field(description="The number of stars given in the rating.")
    count: int = Field(description="The number of reviews that gave this star rating.")

class HotelDetails(BaseModel):
    type_: str = Field(description="The type of listing, in this case, 'hotel'.")
    name: str = Field(description="The name of the hotel.")
    description: str = Field(description="A brief description of the hotel.")
    gps_coordinates: GpsCoordinates = Field(description="The GPS coordinates (latitude and longitude) of the hotel.")
    check_in_time: str = Field(description="The check-in time at the hotel.")
    check_out_time: str = Field(description="The check-out time at the hotel.")
    rate_per_night: Rate = Field(description="Details about the rate per night, including lowest rates and rates before taxes and fees.")
    total_rate: Rate = Field(description="Total rate details for the stay, including lowest rates and rates before taxes and fees.")
    nearby_places: List[NearbyPlace] = Field(description="A list of places nearby the hotel, with available transportation options.")
    hotel_class: str = Field(description="The class of the hotel (e.g., 1-star tourist hotel).")
    overall_rating: float = Field(description="The overall rating of the hotel.")
    reviews: int = Field(description="The total number of reviews for the hotel.")
    ratings: List[Rating] = Field(description="A breakdown of ratings by star count.")
    location_rating: float = Field(description="The rating of the hotel's location.")
    amenities: List[str] = Field(description="A list of amenities provided by the hotel (e.g., Free Wi-Fi, Pet-friendly).")

In [214]:
class Airport(BaseModel):
    name: str = Field(description="The name of the airport.")
    airport_id: str = Field(description="The unique identifier (IATA code) of the airport.")
    time: str = Field(description="The scheduled time of departure or arrival.")

class Flight(BaseModel):
    departure_airport: Airport = Field(description="The departure airport details.")
    arrival_airport: Airport = Field(description="The arrival airport details.")
    duration: int = Field(description="Duration of the flight in minutes.")
    airline: str = Field(description="The airline operating the flight.")
    travel_class: str = Field(description="The travel class of the flight (e.g., Economy, Business).")
    flight_number: str = Field(description="The flight number assigned by the airline.")

class FlightDetails(BaseModel):
    flights: List[Flight] = Field(description="A list of flight segments included in this itinerary.")
    total_duration: int = Field(description="Total duration of the flight(s) in minutes.")
    price: int = Field(description="The price of the flight in the specified currency.")

class Activity(BaseModel):
    """Activity"""
    time: datetime.datetime = Field(description='Approximate time of the activicty. The format should be YYYY-MM-DD HH:MM:SS')
    place: str              = Field(description='Name of the place where is located the activicty')
    description: str        = Field(description='Description of the activicty')

class DayPlan(BaseModel):
    """Plan of a day in the travel plan"""
    date: datetime.date          = Field(description="Date of the plan.")
    activities: List[Activity]   = Field(description="List of activities to do in the day plan.")

class TravelPlan(BaseModel):
    """Travel plan"""
    day_plans: List[DayPlan] = Field(description="List of day plans activities.")


class EnrichedTravelPlan(BaseModel):
    activities_travel_plan: TravelPlan     = Field(description="Travel plan for the activities to do during the holiday.")
    outbound_flight_details: FlightDetails = Field(description="Detailed description of the outbound flight.") 
    return_flight_details: FlightDetails   = Field(description="Detailed description of the return flight.") 
    hotels_details: List[HotelDetails]     = Field(description="Detailed description of the accomodations for the travel.")  


In [215]:
outbound_flight_options = json.dumps([
    FlightDetails(**eval(str(option).replace("'id': ", "'airport_id': "))).dict()
    for option in search_flight_result['outbound_input']['best_flights'] + search_flight_result['outbound_input']['other_flights']
])

return_flight_options = json.dumps([
    FlightDetails(**eval(str(option).replace("'id': ", "'airport_id': "))).dict()
    for option in search_flight_result['return_input']['best_flights'] + search_flight_result['return_input']['other_flights']
])

accomodation_options = json.dumps([
    [
        HotelDetails(**eval(str(hotel_queries[0]['properties'][0]).replace('type', 'type_'))).dict()
        for hotel_option in hotel_query['properties']
    ]
    for hotel_query in hotel_queries 
])

In [216]:
return_flight_options

'[{"flights": [{"departure_airport": {"name": "Paris Charles de Gaulle Airport", "airport_id": "CDG", "time": "2024-11-02 06:05"}, "arrival_airport": {"name": "Milan Malpensa Airport", "airport_id": "MXP", "time": "2024-11-02 07:35"}, "duration": 90, "airline": "easyJet", "travel_class": "Economy", "flight_number": "U2 4541"}], "total_duration": 90, "price": 36}, {"flights": [{"departure_airport": {"name": "Paris Charles de Gaulle Airport", "airport_id": "CDG", "time": "2024-11-02 09:05"}, "arrival_airport": {"name": "Milan Malpensa Airport", "airport_id": "MXP", "time": "2024-11-02 10:35"}, "duration": 90, "airline": "easyJet", "travel_class": "Economy", "flight_number": "U2 3806"}], "total_duration": 90, "price": 36}, {"flights": [{"departure_airport": {"name": "Paris Charles de Gaulle Airport", "airport_id": "CDG", "time": "2024-11-02 16:55"}, "arrival_airport": {"name": "Milan Malpensa Airport", "airport_id": "MXP", "time": "2024-11-02 18:20"}, "duration": 85, "airline": "Air Franc

In [217]:
user_input

{'destination': 'Paris',
 'departure_date': '2024-10-31',
 'return_date': '2024-11-02',
 'travel_text': "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night."}

In [218]:
llm_parser = ChatOpenAI(model="gpt-4o")

prompt_template_parse = PromptTemplate(
    input_variables=[
        "destination", "departure_date",
        "return_date", "travel_text", 
        "travel_plan", "outbound_flights_options",
        "return_flights_options", "accomodation_options"
    ],
    template=GPTRAVEL_PROMPT_PARSER_TEMPLATE
)

params_parse = user_input | {
    "travel_plan": travel_plan_output.json(),
    "outbound_flights_options": outbound_flight_options,
    "return_flights_options": outbound_flight_options,
    "accomodation_options": accomodation_options
}


params_parse

{'destination': 'Paris',
 'departure_date': '2024-10-31',
 'return_date': '2024-11-02',
 'travel_text': "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night.",
 'travel_plan': '{"day_plans": [{"date": "2024-10-31", "activities": [{"time": "2024-10-31T10:00:00", "place": "Eiffel Tower", "description": "Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."}, {"time": "2024-10-31T12:00:00", "place": "Champs-\\u00c9lys\\u00e9es", "description": "Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}, {"time": "2024-10-31T15:00:00", "place": "Louvre Museum", "description": "Visit the world\'s largest art museum and a historic monument in Paris. Don\'t miss the Mona Lisa and the Ve

In [219]:
structured_llm_parser = llm_parser.with_structured_output(EnrichedTravelPlan)

In [220]:
print(prompt_template_parse.format(**params_parse))


You are an intelligent travel planner.

The user is interested to travel to Paris between 2024-10-31 and 2024-11-02. 
In addition to that the user suggested this for his/her travel: "I wanna visit Paris with my girlfriend in a romantic weekend. I'd love to impress her but at the same time I'd love to visit the best places of the city. I would like to sleep also in a good place, maximum 300 euros per night.".

In the previous iterations, you did the following things:

- Created the following travel plan: {"day_plans": [{"date": "2024-10-31", "activities": [{"time": "2024-10-31T10:00:00", "place": "Eiffel Tower", "description": "Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."}, {"time": "2024-10-31T12:00:00", "place": "Champs-\u00c9lys\u00e9es", "description": "Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}, {"time": "2024-10-31T15:0

In [221]:
enriched_travel_plan_output = structured_llm_parser.invoke(
    prompt_template_parse.format(**params_parse)
)

In [222]:
print(enriched_travel_plan_output.json())

{"activities_travel_plan":{"day_plans":[{"date":"2024-10-31","activities":[{"time":"2024-10-31T18:00:00","place":"Eiffel Tower","description":"Start your romantic weekend with a visit to the iconic Eiffel Tower. Take an elevator ride to the top for a breathtaking view of Paris."},{"time":"2024-10-31T20:00:00","place":"Champs-Élysées","description":"Stroll down the famous avenue, stopping by luxury boutiques and cafes. Enjoy some shopping and people-watching."}]},{"date":"2024-11-01","activities":[{"time":"2024-11-01T10:00:00","place":"Louvre Museum","description":"Visit the world's largest art museum and a historic monument in Paris. Don't miss the Mona Lisa and the Venus de Milo."},{"time":"2024-11-01T12:00:00","place":"Seine River","description":"Take a romantic evening cruise on the Seine River. Enjoy the beautiful views of the city's landmarks illuminated at night."}]},{"date":"2024-11-02","activities":[{"time":"2024-11-02T10:00:00","place":"Montmartre","description":"Explore the c